# Architecture et particularités du transformer

Jusqu'à présent, nous avons regardé l'architecture décodeur du *transformer* et uniquement la partie avec la *masked multi-head self-attention*. Dans cette partie, nous allons voir l'intuition derrière le block encodeur et la *multi-head cross-attention*.

## Particularité du texte

Avant de présenter le block encodeur, il est nécessaire de bien comprendre pourquoi nous n'avions besoin que du block décodeur les notebooks précédents. Dans la figure suivante, l'encadré rouge indique la partie du modèle que nous avons utilisé (sans la seconde *multi-head attention*).

<img src="images/decoderblock.png" alt="arllm" width="400"/>

Le texte a la particularité de se lire de gauche à droite. Il est donc logique de supposer qu'un modèle qui génére du texte va aussi prendre les informations de gauche à droite. C'est ce qu'on appelle un [modèle autoregressif](https://deepgenerativemodels.github.io/notes/autoregressive/). On va prédire le *token* suivant compte tenu des *tokens* précédents (compris dans le contexte).  
C'est pourquoi, lors de l'entraînement, on utilise un masque pour ne pas "voir" les *tokens* situés après le *token* actuel.

Cependant, cette approche n'a du sens que dans des cas de "génération de gauche à droite". Il y a énormement de cas où l'approche autoregressive n'est pas l'approche optimale. Parmis ces cas, on retrouve certaines approches de NLP comme la traduction ou l'analyse des émotions mais également des approches de vision. Actuellement, l'architecture du *transformer* a fait ces preuves dans la majorité des domaines utilisant le deep learning et dans bon nombre de cas, il ne s'agit pas de l'approche autogressive.


<img src="images/ar-llm.png" alt="arllm" width="500"/> 

AR-LLM signifie *auto regressive large language model*. 

## Le block encodeur

Le *block* encodeur est le *block* de gauche representé sur la figure de l'architecture (encadré en rouge à nouveau).

<img src="images/encoderblock.png" alt="arllm" width="400"/>

La seule différence avec le *block* utilisé précedemment est que la couche *multi-head attention* n'est pas *masked*. Pour faire l'analogie avec les notebooks précédents, cela signifie qu'on utilise une matrice complète au lieu d'une matrice triangulaire inférieure pour le calcul de l'*attention*.

Concrétement, cela signifie que chaque *token* de la séquence d'entrée peut interragir avec tous les autres *tokens* (situés avant ou après). Dans des cas comme l'analyse de sentiments, c'est l'approche privilégiée car on a une séquence connue en entrée et on cherche à prédire un label (positif, négatif ou neutre).  

**Point sur l'analyse de sentiments** : En NLP, l'analyse de sentiments consiste à donner au modèle un texte et lui demander de ressortir le sentiment associé à ce texte. Par exemple, pour une critique de film, on veut que le modèle prédise négatif pour la critique "Ce film est un vrai navet" et positif pour la critique "Pour moi, c'est le meilleur film de tout les temps".  

**Point important à considérer** : Pour prédire le sentiment associé à une phrase, on a besoin de ne passer qu'une seule fois dans le *transformer*. Pour la génération de texte, on doit faire appel au modèle après chaque token généré (donc on passe 10 fois dans le modèle pour générer 10 tokens).

Les modèles *transformer* basés uniquement sur le *block* encodeur ont de nombreuse utilisations en plus de la détection de sentiments : Détection de spam, classification de documents, extraction d'entitées nommées et recommendation de contenu. Nous verrons aussi que pour le traitement d'images, on utilise une variante du *transformer* pouvant s'apparenter à un encodeur. Pour résumer, on pourrait dire que le *block* *encoder* est adapté aux tâches de classification (détection et segmentation également pour les images).

## Architecture complète avec cross-attention

Il nous reste à comprendre l'utilité de l'architecture complète. Pour rappel, l'article ["Attention Is All You Need"](https://arxiv.org/pdf/1706.03762) qui introduit le transformer est un article de *traduction* automatique.  

Analysons le problème de traduction avant de comprendre le fonctionnement de l'architecture. En traduction, on dispose d'un texte dans une langue et on veut générer le même texte dans une autre langue. On a donc d'une part, une partie génération (donc décodeur) et d'autre part, une partie d'encodage de l'information que l'on a a disposition. 

Il faut imaginer que la partie décodeur qui génère les *tokens* va le faire avec d'une part les *tokens* générés précedemment mais également en interrogeant la partie encodé via la couche de *cross-attention*.  

### Formalisation mathématique

L'*encoder* du *transformer* prend en entrée la séquence source $x$ et produit une représentation contextuelle pour chaque *token* de cette séquence :
$E = \text{Encoder}(x)$   
Ici, $E$ est une matrice de représentations contextuelles pour la séquence source $x$. Chaque ligne de $E$ correspond à une représentation contextuelle $e_i$ pour le *token* $x_i$.

Le *decoder* du *transformer* prend en entrée les représentations contextuelles $E$ de l'*encoder* et génère la séquence cible $y$. À chaque étape de génération, le *decoder* produit un *token* $y_t$ en se basant sur les *tokens* générés précédemment et en interrogeant l'*encoder* via la couche de c*ross-attention* :  
$y_t = \text{Decoder}(y_{<t}, E) $   
Ici, $y_{<t}$ représente les *tokens* générés précédemment jusqu'à l'étape $t-1$. Le processus de génération utilise à la fois le *self-attention* (pour capturer les dépendances séquentielles dans la séquence cible) et la *cross-attention* (pour incorporer des informations de l'*encoder* $E$).

La *cross-attention* permet au *decoder* de consulter les représentations contextuelles $E$ de l'*encoder* pour obtenir des informations pertinentes lors de la génération de chaque *token* $y_t$. Elle est calculée comme :  
$ \text{Cross-Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V $  
où $Q$ (*query*) sont les embeddings des tokens générés précédemment par le *decoder*, $K$(*key*) et $V$(*value*) sont les *embeddings* de l'*encoder* $E$, et $d_k$ est la dimension des *embeddings* $K$ pour la normalisation.

En combinant ces éléments, le modèle *transformer* peut efficacement traduire une séquence de *tokens* d'une langue source à une langue cible en utilisant des mécanismes d'attention et des *postionnal embedding* pour maintenir l'ordre séquentiel et capturer les dépendances à long terme.

**Notes** : Le modèle décodeur essaye de générer un *token* pertinent compte tenu des *tokens* de l'encodeur. Il va émettre une requête (*query*) et regarder les clés (*key*) et valeurs (*value*) transmises par l'encodeur via la couche de *cross-attention*. Dans un deuxième temps, il va émettre une requête, une clé et une valeur pour trouver un *token* cohérent par rapport aux *tokens* qu'il a généré précedemment via la couche de *self-attention*.

### Exemples d'utilisation

L'architecture complète va être utilisée dans des cas où l'on cherche à générer un texte à partir d'un autre texte. Les cas d'applications les plus communs sont : la traduction, le résumé de texte, correction automatique et génération de texte guidée. 

**Notes** : Vous avez sans doute constaté que ChatGpt est capable de faire de la traduction et du résumé de texte. En effet, le modèle est tellement puissant qu'il arrive à réaliser des tâches qui sont en théorie difficiles pour lui. Il est entraîné à prédire le prochain *token* et en y reflechissant un peu, on peut facilement imager que la tâche de prédiction du prochain *token* permette de réaliser toutes les tâches de NLP.   
Cependant, pour des modèles plus restreints, il faut mieux s'en tenir une architecture adaptée à notre problème. 